In [1]:
from sympy import Matrix, MatrixSymbol, Poly, nsimplify, shape
from sympy.solvers import solve
from scipy.optimize import linprog

## Create the Adjacency matrix

In [2]:
adj=[[0,0,0,0,1,1,0,0,0,0,0,0],
[0,0,0,0,0,0,1,0,1,0,0,0],
[0,0,0,0,0,0,0,0,0,1,0,1],
[0,0,0,0,0,0,0,1,0,0,1,0],
[1,0,0,0,0,1,0,0,0,0,0,1],
[1,0,0,0,1,0,0,0,0,0,1,0],
[0,1,0,0,0,0,0,1,0,1,0,0],
[0,0,0,1,0,0,1,0,1,0,0,0],
[0,1,0,0,0,0,0,1,0,0,0,1],
[0,0,1,0,0,0,1,0,0,0,1,0],
[0,0,0,1,0,1,0,0,0,1,0,0],
[0,0,1,0,1,0,0,0,1,0,0,0]]

In [3]:
M = Matrix(adj)

## Define the constraints for doubly stochastic matrices

In [4]:
# 12 x 12 symbolic matrix
D = MatrixSymbol('x', 12, 12).as_explicit()

e = Matrix.ones(12,1)

For a matrix $D \in R^{n\times n}$ to be doubly stochastic, the sum of entries within each row and within each column must be equal to 1. Give the matrix $e = [1, 1, \dots, 1]^T \in R^{n \times 1}$, these constraints can be expressed as

$$
De=D^Te=e.
$$

Furthermore, we wish to find a doubly stochastic matrix that commutes with our adjacency matrix $M$. This constraint can be expressed as

$$
MD-DM=\bf{0}.
$$

In [5]:
doubly_stochastic_soln = solve([(D*e-e),D.transpose()*e-e, M*D-D*M],D, positive=True)

In [6]:
# The matrix obtained by replacing dependent variables with the equations that define their dependencies
D_dep = D.xreplace(doubly_stochastic_soln)
D_dep

Matrix([
[                            -4*x[11, 1] - 7*x[11, 2] - 3*x[11, 3] - 5*x[11, 5] - 4*x[11, 6] - 5*x[11, 7] + x[11, 9] - x[11, 11] + 2,                                   -x[11, 1] + 2*x[11, 2] + x[11, 3] + 2*x[11, 6] + 3*x[11, 7] - x[11, 8] - 2*x[11, 10],            5*x[11, 1] + 6*x[11, 2] + 3*x[11, 3] + 6*x[11, 5] + 2*x[11, 6] + 2*x[11, 7] - 2*x[11, 9] + x[11, 10] + x[11, 11] - 1,                                                         -x[11, 2] - x[11, 3] - x[11, 5] + x[11, 8] + x[11, 9] + x[11, 10],                                          -2*x[11, 1] - 3*x[11, 2] - 2*x[11, 3] - x[11, 5] - 3*x[11, 6] - 3*x[11, 7] - x[11, 11] + 1,                        3*x[11, 1] + 8*x[11, 2] + 2*x[11, 3] + 7*x[11, 5] + x[11, 6] + 2*x[11, 7] - 4*x[11, 8] - 5*x[11, 9] - x[11, 10], 4*x[11, 1] + 3*x[11, 2] + 2*x[11, 3] + 3*x[11, 5] + 2*x[11, 6] + 2*x[11, 7] + 3*x[11, 8] + x[11, 9] + 3*x[11, 10] + 2*x[11, 11] - 2,   6*x[11, 1] + 4*x[11, 2] + 3*x[11, 3] + 4*x[11, 5] + 2*x[11, 6] + x[11, 7] + 3*x[1

## Find an explicit solution
With the dependencies defined, we now wish to find an explicit solution

The first step is find the _free_ variables, i.e. the variables with no dependencies.

In [7]:
free = sorted(list(set(D)-set(doubly_stochastic_soln.keys())), key = lambda a: int(str(a).split()[1][:-1]))
free

[x[11, 1],
 x[11, 2],
 x[11, 3],
 x[11, 5],
 x[11, 6],
 x[11, 7],
 x[11, 8],
 x[11, 9],
 x[11, 10],
 x[11, 11]]

Then we create the matrix of coefficients for the linear system that ensures each entry in $D$ is between $0$ and $1$. Specifically, we observe that $a_1x_1 + a_2x_2 + \dots + a_mx_m + k\leq 1$ is equivalent to $a_1x_1 + a_2x_2 + \dots + a_mx_m \leq 1-k$, and $a_1x_1 + a_2x_2 + \dots + a_mx_m + k\geq 0$ is equivalent to $-a_1x_1 - a_2x_2 - \dots - a_mx_m\leq k$.

In [8]:
pos_coeffs = Matrix([[Poly(d, free).coeff_monomial(var) for var in free] for d in D_dep])
neg_coeffs = -1*pos_coeffs
upper_bound = Matrix([[1-Poly(d,free).coeff_monomial(1)] for d in D_dep])
lower_bound = Matrix([[Poly(d,free).coeff_monomial(1)] for d in D_dep])

constraint_coeffs = Matrix.vstack(pos_coeffs, neg_coeffs)
constraint_bounds = Matrix.vstack(upper_bound, lower_bound)

To find an explicit solution, we will use a linear programming solver. This solver requires a _target_ function; a function the solver tries to minimise over the feasible region our constraints have defined. For our purpose, we don't care what this function is, so we arbitrarily set it $0$.

In [9]:
c = [0 for d in free]

Now we have everything necessary to find a solution:

In [10]:
explicit_soln = linprog(c, constraint_coeffs, constraint_bounds, bounds=(0,1))

# Substitute values into matrix
subst = {f: nsimplify(explicit_soln.x[i]) for (i,f) in enumerate(free)}
D_explicit = D_dep.xreplace(subst)
D_explicit

Matrix([
[1/7, 2/7, 2/7, 2/7,   0,   0,   0,   0,   0,   0,   0,   0],
[2/7, 1/7, 2/7, 2/7,   0,   0,   0,   0,   0,   0,   0,   0],
[2/7, 2/7, 1/7, 2/7,   0,   0,   0,   0,   0,   0,   0,   0],
[2/7, 2/7, 2/7, 1/7,   0,   0,   0,   0,   0,   0,   0,   0],
[  0,   0,   0,   0,   0, 1/7, 1/7, 1/7, 1/7, 1/7, 1/7, 1/7],
[  0,   0,   0,   0, 1/7,   0, 1/7, 1/7, 1/7, 1/7, 1/7, 1/7],
[  0,   0,   0,   0, 1/7, 1/7,   0, 1/7, 1/7, 1/7, 1/7, 1/7],
[  0,   0,   0,   0, 1/7, 1/7, 1/7,   0, 1/7, 1/7, 1/7, 1/7],
[  0,   0,   0,   0, 1/7, 1/7, 1/7, 1/7,   0, 1/7, 1/7, 1/7],
[  0,   0,   0,   0, 1/7, 1/7, 1/7, 1/7, 1/7,   0, 1/7, 1/7],
[  0,   0,   0,   0, 1/7, 1/7, 1/7, 1/7, 1/7, 1/7,   0, 1/7],
[  0,   0,   0,   0, 1/7, 1/7, 1/7, 1/7, 1/7, 1/7, 1/7,   0]])

As a check, we show that $MD=DM$

In [11]:
M*D_explicit==D_explicit*M

True

## Make this a function

In [12]:
def find_DS_commutor(M):
    s = shape(M)[0]
    D = MatrixSymbol('x', s, s).as_explicit()
    e = Matrix.ones(s,1)
    
    doubly_stochastic_soln = solve([(D*e-e),D.transpose()*e-e, M*D-D*M],D, positive=True)
    D_dep = D.xreplace(doubly_stochastic_soln)
    free = sorted(list(set(D)-set(doubly_stochastic_soln.keys())), key = lambda a: int(str(a).split()[1][:-1]))
    
    pos_coeffs = Matrix([[Poly(d, free).coeff_monomial(var) for var in free] for d in D_dep])
    neg_coeffs = -1*pos_coeffs
    upper_bound = Matrix([[1-Poly(d,free).coeff_monomial(1)] for d in D_dep])
    lower_bound = Matrix([[Poly(d,free).coeff_monomial(1)] for d in D_dep])

    constraint_coeffs = Matrix.vstack(pos_coeffs, neg_coeffs)
    constraint_bounds = Matrix.vstack(upper_bound, lower_bound)
    c = [0 for d in free]
    
    explicit_soln = linprog(c, constraint_coeffs, constraint_bounds, bounds=(0,1))

    # Substitute values into matrix
    subst = {f: nsimplify(explicit_soln.x[i]) for (i,f) in enumerate(free)}
    return D_dep.xreplace(subst)

In [13]:
adj_2 = [[0,0,1,1,0,0,0,0,1,0],
[0,0,0,0,1,1,0,0,0,1],
[1,0,0,0,0,0,1,0,1,0],
[1,0,0,0,0,0,0,1,0,1],
[0,1,0,0,0,0,1,0,0,1],
[0,1,0,0,0,0,0,1,1,0],
[0,0,1,0,1,0,0,1,1,0],
[0,0,0,1,0,1,1,0,0,1],
[1,0,1,0,0,1,1,0,0,1],
[0,1,0,1,1,0,0,1,1,0]]
M_2 = Matrix(adj_2)

In [14]:
find_DS_commutor(M_2)

Matrix([
[1/3, 2/3,   0,   0,   0,   0,   0,   0,   0,   0],
[2/3, 1/3,   0,   0,   0,   0,   0,   0,   0,   0],
[  0,   0,   0, 1/3, 1/3, 1/3,   0,   0,   0,   0],
[  0,   0, 1/3,   0, 1/3, 1/3,   0,   0,   0,   0],
[  0,   0, 1/3, 1/3,   0, 1/3,   0,   0,   0,   0],
[  0,   0, 1/3, 1/3, 1/3,   0,   0,   0,   0,   0],
[  0,   0,   0,   0,   0,   0, 1/3, 2/3,   0,   0],
[  0,   0,   0,   0,   0,   0, 2/3, 1/3,   0,   0],
[  0,   0,   0,   0,   0,   0,   0,   0, 1/3, 2/3],
[  0,   0,   0,   0,   0,   0,   0,   0, 2/3, 1/3]])